In [21]:
# Khai báo các thư viện cần thiết

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pickle
import numpy as np
import torch.optim as optim
import os

In [22]:
# Khai báo tên model
while True:
    model_name = input("Enter model name: ")
    if model_name == "":
        print("Model name cannot be empty. Please enter a valid name.")
    else:
        if not os.path.exists(model_name):
            print("Model is not exist. Please enter a valid name.")
        else:
            break

In [23]:
# Mạng Nơ ron 
class MazeNetCombined(nn.Module):
    def __init__(self):
        super(MazeNetCombined, self).__init__()
        
        # Quan sát cục bộ
        self.conv1_local = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2_local = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3_local = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4_local = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        # Quan sát toàn cục
        self.conv1_global = nn.Conv2d(1, 32, kernel_size=2, stride=2, padding=0)
        self.conv2_global = nn.Conv2d(32, 64, kernel_size=2, stride=2, padding=0)

        # Xử lý vị trí
        self.fc_position = nn.Linear(2, 32)  # Vị trí hiện tại

        # Tầng Fully Connected cuối cùng
        self.fc1 = nn.LazyLinear(256)
        self.fc2 = nn.LazyLinear(128) 
        self.dropout_fc = nn.Dropout(p=0.5)  # Dropout trước tầng FC3
        self.fc3 = nn.LazyLinear(4)  # Đầu ra cho 4 hành động (lên, xuống, trái, phải)
    
    def forward(self, local_obs, global_obs, position):
        # Xử lý local_obs
        x_local = F.relu(self.conv1_local(local_obs))
        x_local = F.relu(self.conv2_local(x_local))
        x_local = F.relu(self.conv3_local(x_local))
        x_local = F.relu(self.conv4_local(x_local))
        x_local = x_local.view(x_local.size(0), -1)
        
        # Xử lý global_obs
        x_global = F.relu(self.conv1_global(global_obs))
        x_global = F.relu(self.conv2_global(x_global))
        x_global = x_global.view(x_global.size(0), -1)

        # Xử lý vị trí hiện tại
        x_position = F.relu(self.fc_position(position))
        
        # Kết hợp tất cả
        x = torch.cat((x_local, x_global, x_position), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout_fc(x)  # Dropout trước FC3
        x = self.fc3(x)

        return x

In [24]:
# Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity, path, save_prob=0.1):
        """
        Khởi tạo Replay Buffer.
        Args:
            capacity (int): Sức chứa của buffer chính.
            path (str): Đường dẫn để load/lưu buffer (saved_buffer).
            save_prob (float): Xác suất lưu trải nghiệm mới vào saved_buffer.
        """
        self.capacity = capacity
        self.buffer = []
        self.saved_buffer = []  # Buffer lưu từ tệp
        self.path = path
        self.save_prob = save_prob
        
        # Tải replay buffer đã lưu nếu tồn tại
        self._load_saved_buffer()

    def _load_saved_buffer(self):
        """Tải saved_buffer từ tệp."""
        try:
            with open(self.path, 'rb') as f:
                self.saved_buffer = pickle.load(f)
            with open(self.path, 'rb') as f:
                self.buffer = pickle.load(f)
            print(f"Saved buffer loaded successfully from {self.path}.")
        except FileNotFoundError:
            print(f"No saved buffer found at {self.path}. Starting with an empty saved buffer.")
        except Exception as e:
            print(f"Error loading saved buffer: {e}. Starting with an empty saved buffer.")

    def save_saved_buffer(self):
        """Lưu saved_buffer vào tệp tại path."""
        try:
            with open(self.path, 'wb') as f:
                pickle.dump(self.saved_buffer, f)
            print(f"Saved buffer successfully saved to {self.path}.")
        except Exception as e:
            print(f"Error saving saved buffer: {e}.")

    def push(self, experience):
        """Thêm một trải nghiệm vào buffer chính và có xác suất thêm vào saved_buffer."""
        self.buffer.append(experience)
        
        # Với xác suất save_prob, thêm trải nghiệm vào saved_buffer
        if np.random.rand() < self.save_prob:
            if len(self.saved_buffer) >= self.capacity:
                # Nếu saved_buffer đã đầy, xóa trải nghiệm cũ nhất
                self.saved_buffer.pop(0)
            self.saved_buffer.append(experience)

    def sample(self, batch_size):
        """Trích xuất batch mẫu từ buffer chính."""
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        batch = [self.buffer[idx] for idx in indices]
        # Tách dữ liệu thành các phần riêng biệt
        local_obs, global_obs, position, actions, rewards, next_local_obs, next_global_obs, next_position, dones = zip(*batch)
        return (np.array(local_obs), 
                np.array(global_obs), 
                np.array(position), 
                np.array(actions), 
                np.array(rewards), 
                np.array(next_local_obs), 
                np.array(next_global_obs), 
                np.array(next_position),
                np.array(dones))

    def __len__(self):
        return len(self.buffer)

In [25]:
# Cập nhật model
def update_model(policy_net, target_net, replay_buffer, optimizer, batch_size, gamma, device):
    """
    Cập nhật mô hình chính (policy network) cho mạng có đầu vào đa dạng (local_obs, global_obs, position).

    Args:
    - policy_net (nn.Module): Mạng chính dự đoán giá trị Q(s, a).
    - target_net (nn.Module): Mạng mục tiêu dùng để tính Q_target.
    - replay_buffer (ReplayBuffer): Bộ nhớ replay chứa các kinh nghiệm (local_obs, global_obs, position, action, reward, next_local_obs, next_global_obs, next_position, done).
    - optimizer (torch.optim.Optimizer): Trình tối ưu hóa (Adam, SGD, ...).
    - batch_size (int): Kích thước batch mẫu từ replay buffer.
    - gamma (float): Hệ số chiết khấu (discount factor).
    - device (torch.device): Thiết bị thực thi (CPU hoặc GPU).

    Returns:
    - loss (float): Giá trị mất mát (loss) sau khi cập nhật.
    """
    # Kiểm tra nếu replay buffer chưa đủ dữ liệu
    if len(replay_buffer) < batch_size:
        return None

    # 1. Lấy mẫu từ replay buffer
    batch = replay_buffer.sample(batch_size)
    (local_obs, global_obs, position, actions, rewards, 
     next_local_obs, next_global_obs, next_position, dones) = batch

    # 2. Chuyển đổi dữ liệu sang Tensor và đưa vào thiết bị (CPU/GPU)
    local_obs = torch.tensor(local_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh cho CNN
    global_obs = torch.tensor(global_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh
    position = torch.tensor(position, dtype=torch.float32).to(device)
    actions = torch.tensor(actions, dtype=torch.long).to(device)
    rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
    next_local_obs = torch.tensor(next_local_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh
    next_global_obs = torch.tensor(next_global_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh
    next_position = torch.tensor(next_position, dtype=torch.float32).to(device)
    dones = torch.tensor(dones, dtype=torch.float32).to(device)

    # 3. Dự đoán Q(s, a) từ policy_net
    q_values = policy_net(local_obs, global_obs, position)  # Đầu ra: (batch_size, num_actions)
    q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)  # Lấy giá trị Q(s, a) cho hành động đã thực hiện

    # 4. Tính toán Q_target bằng target_net
    with torch.no_grad():
        next_q_values = target_net(next_local_obs, next_global_obs, next_position)  # Dự đoán Q(s', a') từ target_net
        max_next_q_values = next_q_values.max(1)[0]  # Lấy giá trị lớn nhất Q(s', a')
        q_targets = rewards + gamma * max_next_q_values * (1 - dones)  # Hàm Bellman

    # 5. Tính hàm mất mát   
    loss = F.mse_loss(q_values, q_targets)

    # 6. Tối ưu hóa mô hình
    optimizer.zero_grad()  # Xóa gradient cũ
    loss.backward()  # Lan truyền ngược (backpropagation)
    optimizer.step()  # Cập nhật trọng số

    return loss.item()


In [26]:
# Đồng bộ với mạng mục tiêu
def sync_target_network(q_network, target_network):
    target_network.load_state_dict(q_network.state_dict())

In [27]:
# Phương thức lưu, tải trọng số của mô hình

# Lưu trọng số của mô hình
def save_model(model, path):
    torch.save(model, path)
    print(f"Model saved to: {path}")
    
# Tải trọng số của mô hình
def load_model(path, device = "cuda"):
    policy_model = torch.load(path, map_location=device, weights_only=False)  # Tải mô hình từ file
    target_model = policy_model
    target_model.eval() # Đặt target_model ở chế độ đánh giá
    print(F"Model loaded from: {path}")
    return policy_model, target_model

In [28]:
# Khởi tạo các siêu tham số

# Thiết bị thực thi (CPU hoặc GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

# Đọc training info từ file
with open(model_name + "/training_info.pkl", "rb") as f:
    training_info = pickle.load(f)
gamma = training_info["gamma"]  # Hệ số chiết khấu
learning_rate = training_info["learning_rate"]  # Tốc độ học
weight_decay = training_info["weight_decay"]  # Hệ số điều chỉnh trọng số

# Hyperparameters
batch_size = 256  # Kích thước batch khi lấy mẫu từ replay buffer
max_episode = 10000             # Số episode tối đa

# Tải model từ file 
policy_net, target_net = load_model(model_name + "/model.pth", device)

# Khởi tạo replay buffer
replay_buffer = ReplayBuffer(20000, model_name + "/replay_buffer.pkl", 0.15)  

# Optimizer
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate, weight_decay = weight_decay)

print("Initialization complete!")

Training on: cuda
Model loaded from: model03/model.pth
Saved buffer loaded successfully from model03/replay_buffer.pkl.
Initialization complete!


In [29]:
# Huấn luyện mô hình trên replay buffer
for episode in range(max_episode):
    print(f"Episode {episode + 1}/{max_episode}")
    loss = update_model(policy_net, target_net, replay_buffer, optimizer, batch_size, gamma, device)
    
    if loss is not None:
        print(f"Loss: {loss:.4f}")
    else:
        print("Replay buffer not enough data for training.")

    # Đồng bộ hóa mạng mục tiêu
    if episode % 100 == 0:
        sync_target_network(policy_net, target_net)

Episode 1/10000
Loss: 64.5891
Episode 2/10000
Loss: 66.0822
Episode 3/10000
Loss: 82.0992
Episode 4/10000
Loss: 77.2737
Episode 5/10000
Loss: 108.4819
Episode 6/10000
Loss: 74.7802
Episode 7/10000
Loss: 110.2836
Episode 8/10000
Loss: 78.2179
Episode 9/10000
Loss: 74.8251
Episode 10/10000
Loss: 69.1663
Episode 11/10000
Loss: 91.9433
Episode 12/10000
Loss: 75.4720
Episode 13/10000
Loss: 73.3837
Episode 14/10000
Loss: 79.3876
Episode 15/10000
Loss: 78.6154
Episode 16/10000
Loss: 71.5784
Episode 17/10000
Loss: 67.7028
Episode 18/10000
Loss: 66.4156
Episode 19/10000
Loss: 59.4209
Episode 20/10000
Loss: 64.1273
Episode 21/10000
Loss: 56.5736
Episode 22/10000
Loss: 59.4313
Episode 23/10000
Loss: 78.5527
Episode 24/10000
Loss: 58.3580
Episode 25/10000
Loss: 44.8672
Episode 26/10000
Loss: 56.3179
Episode 27/10000
Loss: 52.1420
Episode 28/10000
Loss: 54.5544
Episode 29/10000
Loss: 65.6127
Episode 30/10000
Loss: 45.3343
Episode 31/10000
Loss: 49.5855
Episode 32/10000
Loss: 58.9640
Episode 33/1000

In [30]:
# Lưu mô hình
save_model(policy_net, path=model_name + "/model.pth")

Model saved to: model03/model.pth
